In [2]:
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
%matplotlib inline
import xml.etree.ElementTree as ET
import pandas as pd 
import shutil 
import os 
import glob
from PIL import Image

In [3]:
def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        # ymin = int(boxes.find("bndbox/ymin").text)
        # xmin = int(boxes.find("bndbox/xmin").text)
        # ymax = int(boxes.find("bndbox/ymax").text)
        # xmax = int(boxes.find("bndbox/xmax").text)

        ymin = int(float(boxes.find("bndbox/ymin").text))
        xmin = int(float(boxes.find("bndbox/xmin").text))
        ymax = int(float(boxes.find("bndbox/ymax").text))
        xmax = int(float(boxes.find("bndbox/xmax").text))

        list_with_single_boxes = [xmin, ymin, xmax, ymax, filename]
        list_with_all_boxes.append(list_with_single_boxes)

    return list_with_all_boxes


In [34]:
# base code. Augments only single image and xml pair

xmlfile_path = "tick_test copy/1032.xml"
boxes = read_content(xmlfile_path)
original_image = "tick_test copy/1032.jpg"
image = imageio.imread(original_image)

seq = iaa.Sequential([
    # iaa.Rotate((-15, 15)),
    iaa.AddToBrightness((-100, 100)), 
    # iaa.GaussianBlur(sigma=(0.0, 1)), 
    # iaa.MotionBlur(k=10),  
])

boundbox1 = []
bbs = BoundingBoxesOnImage(boundbox1, shape=image.shape)
for list in boxes:
    boundbox1.append(BoundingBox(x1=list[0], x2=list[2], y1=list[1], y2=list[3]))


# augments the bounding boxes 
image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

df1 = pd.DataFrame(columns = ['after.x1', 'after.y1', 'after.x2', 'after.y2'])

# puts all the bounding boxes in one xmlfile into a dataframe
for i in range(len(bbs.bounding_boxes)):
    after = bbs_aug.bounding_boxes[i]
    df1 = df1.append({'after.x1' : after.x1, 'after.y1' : after.y1, 'after.x2' : after.x2, 'after.y2' : after.y2}, ignore_index = True)

parsing = ET.parse(xmlfile_path)
root1 = parsing.getroot()

x = -1
for xmin in root1.iter('xmin'):
    x = x+1
    new_xmin = df1.at[x,'after.x1']
    xmin.text = str(float(new_xmin))
    
    
# parsing.write('output.xml')
parsing.write(xmlfile_path)

y = -1 
for ymin in root1.iter('ymin'):
    y = y+1
    new_ymin = df1.at[y, 'after.y1']
    ymin.text = str(float(new_ymin))
    
# parsing.write('output.xml')
parsing.write(xmlfile_path)


z = -1 
for xmax in root1.iter('xmax'):
    z = z+1 
    new_xmax = df1.at[z, 'after.x2']
    xmax.text = str(float(new_xmax))

# parsing.write('output.xml')
parsing.write(xmlfile_path)


e = -1 
for ymax in root1.iter('ymax'):
    e = e+1 
    new_ymax = df1.at[e, 'after.y2']
    ymax.text = str(float(new_ymax))

# parsing.write('output.xml')
parsing.write(xmlfile_path)


im = Image.fromarray(image_aug)
im.save(original_image)


In [40]:
# Augmenting function

def augment_jpg_xml(original_image): # goes based off of image path 
    xmlfile_path = original_image.removesuffix('.jpg')
    xmlfile_path = xmlfile_path + '.xml'
    boxes = read_content(xmlfile_path)
    image = imageio.imread(original_image)

    
    # seq = iaa.Sequential([
    #     iaa.AddToBrightness((-100, 100)),
    #     # iaa.Resize(416), 
    #     # # iaa.GaussianBlur(sigma=(0.0, 1)), 
    #     # iaa.MotionBlur(k=5),
    #     # iaa.Multiply((0.75, 1.25), per_channel=0.5),
        
    # ])
    

    seq = iaa.Sequential(
        [
            
        iaa.Resize(416),

        iaa.SomeOf((0, 5),
         [  
            iaa.OneOf([
                iaa.Fliplr(),
                iaa.Flipud(),
                iaa.Rotate((-90, 90)),
            ]), 

            iaa.Rotate((-15, 15)),
            iaa.ShearX((-22, 22)),
            iaa.ShearY((-22, 22)),
            iaa.Affine(scale=(0.5, 1.5)), 
            iaa.AddToBrightness((-100, 100)),

            iaa.OneOf([
                iaa.GaussianBlur(sigma=(0.0, 1)), 
                iaa.MotionBlur(k=3),
            ])

        ]), ], random_order=True)


    boundbox1 = []
    bbs = BoundingBoxesOnImage(boundbox1, shape=image.shape)
    for list in boxes:
        boundbox1.append(BoundingBox(x1=list[0], x2=list[2], y1=list[1], y2=list[3]))


    # augments the bounding boxes 
    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

    df1 = pd.DataFrame(columns = ['after.x1', 'after.y1', 'after.x2', 'after.y2'])

    # puts all the bounding boxes in one xmlfile into a dataframe
    for i in range(len(bbs.bounding_boxes)):
        after = bbs_aug.bounding_boxes[i]
        df1 = df1.append({'after.x1' : after.x1, 'after.y1' : after.y1, 'after.x2' : after.x2, 'after.y2' : after.y2}, ignore_index = True)

    parsing = ET.parse(xmlfile_path)
    root1 = parsing.getroot()

    x = -1
    for xmin in root1.iter('xmin'):
        x = x+1
        new_xmin = df1.at[x,'after.x1']
        xmin.text = str(float(new_xmin))
              

    parsing.write(xmlfile_path)

    y = -1 
    for ymin in root1.iter('ymin'):
        y = y+1
        new_ymin = df1.at[y, 'after.y1']
        ymin.text = str(float(new_ymin))
        
 
    parsing.write(xmlfile_path)


    z = -1 
    for xmax in root1.iter('xmax'):
        z = z+1 
        new_xmax = df1.at[z, 'after.x2']
        xmax.text = str(float(new_xmax))


    parsing.write(xmlfile_path)


    e = -1 
    for ymax in root1.iter('ymax'):
        e = e+1 
        new_ymax = df1.at[e, 'after.y2']
        ymax.text = str(float(new_ymax))


    parsing.write(xmlfile_path)


    im = Image.fromarray(image_aug)
    im.save(original_image)





In [38]:
# Augmenting multiple files at once 
for jpgfile in glob.iglob(os.path.join('tick_test copy', "*.jpg")):
    augment_jpg_xml(jpgfile)

KeyboardInterrupt: 

In [39]:
for jpgfile in glob.iglob(os.path.join('test_direct', "*.jpg")):
    augment_jpg_xml(jpgfile)

In [13]:
jpgfile = imageio.imread("tick_test copy/1_0.jpg")

image_aug = seq(image=jpgfile)
from PIL import Image
im = Image.fromarray(image_aug)
im.save("tick_test copy/1_0.jpg")

In [56]:

    seq = iaa.Sequential([
        iaa.Fliplr(0.5), # horizontal flips
        iaa.Crop(percent=(0, 0.1)), # random crops
        # Small gaussian blur with random sigma between 0 and 0.5.
        # But we only blur about 50% of all images.
        iaa.Sometimes(
            0.5,
            iaa.GaussianBlur(sigma=(0, 0.5))
        ),
        # Strengthen or weaken the contrast in each image.
        iaa.LinearContrast((0.75, 1.5)),
        # Add gaussian noise.
        # For 50% of all images, we sample the noise once per pixel.
        # For the other 50% of all images, we sample the noise per pixel AND
        # channel. This can change the color (not only brightness) of the
        # pixels.
        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
        # Make some images brighter and some darker.
        # In 20% of all cases, we sample the multiplier once per channel,
        # which can end up changing the color of the images.
        iaa.Multiply((0.8, 1.2), per_channel=0.2),
        # Apply affine transformations to each image.
        # Scale/zoom them, translate/move them, rotate them and shear them.
        iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-25, 25),
            shear=(-8, 8)
        )
    ], random_order=True) # apply augmenters in random order

In [24]:

def len_folder(folder_path): #tick_test copy/*
    x = 0
    for file in glob.iglob(folder_path + '/*'):
        x = x+1 

    return x

len_folder('all_data/annotations')    

88